In [1]:
import pandas as pd 
import os
pen_info= pd.concat([pd.read_csv(os.path.join('/home/rajesh/work/failed_milkings/events_data',file)) for file in os.listdir('/home/rajesh/work/failed_milkings/events_data') if file.startswith('events')])
failed_df= pd.read_csv('/home/rajesh/work/failed_milkings/final_df_with_days_relative.csv')
pen_info= pen_info[pen_info['PEN'].isin([30,31,32,33,34,35,36])]
pen_info.PEN.value_counts()
pen_info['Date']=pd.to_datetime(pen_info['Date'], format='mixed')

pen_info['Animal Number']=pen_info['ID'].astype(int)

failed_df['Animal Number']= failed_df['Animal Number'].astype(int)

failed_df['Date Time']= pd.to_datetime(failed_df['Date Time'])

failed_df = failed_df.reset_index(drop=True)
pen_info = pen_info.reset_index(drop=True)


import pandas as pd
import numpy as np
from multiprocessing import Pool, cpu_count

# --- Ensure clean datetime and consistent Animal Number type ---
failed_df['Date Time'] = pd.to_datetime(failed_df['Date Time'])
pen_info['Date'] = pd.to_datetime(pen_info['Date'])
failed_df['Animal Number'] = failed_df['Animal Number'].astype(str)
pen_info['Animal Number'] = pen_info['Animal Number'].astype(str)

# --- Create a lookup dict for pen_info grouped by Animal Number ---
pen_groups = {k: v for k, v in pen_info.groupby('Animal Number')}

# --- Function to find the nearest match for one row ---
def find_nearest(row):
    animal = row['Animal Number']
    event_time = row['Date Time']
    
    # If animal not in pen_info, return row unchanged (with NaNs for pen columns)
    if animal not in pen_groups:
        return {**row.to_dict(), 'Matched Date': pd.NaT}
    
    df_animal = pen_groups[animal]
    # Find index of nearest date
    nearest_idx = (df_animal['Date'] - event_time).abs().idxmin()
    nearest_row = df_animal.loc[nearest_idx].to_dict()
    
    # Merge dictionaries
    merged = {**row.to_dict(), **nearest_row}
    merged['Matched Date'] = nearest_row['Date']
    return merged

# --- Run in parallel across all CPU cores ---
num_cores = max(cpu_count() - 1, 1)  # leave one core free
with Pool(num_cores) as pool:
    merged_records = pool.map(find_nearest, [row for _, row in failed_df.iterrows()])

# --- Combine into final DataFrame ---
merged_df = pd.DataFrame(merged_records)

print("✅ Parallel nearest-date merge completed successfully!")
print(merged_df.head())



/tmp/ipykernel_2766/3707399845.py:4: DtypeWarning: Columns (6,49,74,75) have mixed types. Specify dtype option on import or set low_memory=False.
  failed_df= pd.read_csv('/home/rajesh/work/failed_milkings/final_df_with_days_relative.csv')


✅ Parallel nearest-date merge completed successfully!
   Unnamed: 0.2  Unnamed: 0.1 Animal Number  Unnamed: 1  Unnamed: 0  \
0             0             1             3           1           1   
1             1             2             3           2           2   
2             2             3             3           3           3   
3             3             4             3           4           4   
4             4             5             3           5           5   

   Animal Number.1 Gestation  Animal Name  Animal Tag Id  Conductivity LR  \
0                3         3          NaN      6697303.0             66.0   
1                3         3          NaN      6697303.0             65.0   
2                3         3          NaN      6697303.0             67.0   
3                3         3          NaN      6697303.0             64.0   
4                3         3          NaN      6697303.0             63.0   

   ...   DIM       Date            Remark   R  T   B    

In [8]:
merged_df.groupby(['Device Name', 'PEN']).size().reset_index(name='Count').sort_values(['Device Name', 'Count'], ascending=[True, False]).drop_duplicates('Device Name')


,Device Name,PEN,Count
0,Astronaut 101,30.0,24734
6,Astronaut 102,30.0,31616
12,Astronaut 103,30.0,29975
19,Astronaut 104,31.0,29319
25,Astronaut 105,31.0,28176
31,Astronaut 106,31.0,26098
38,Astronaut 107,32.0,29534
44,Astronaut 108,32.0,26463
50,Astronaut 109,32.0,25747
57,Astronaut 110,33.0,39238
